# **Compressing circuit in .qasm files**

In [1]:
import sys
sys.path.insert(0, "../..") # clue is here
from clue import *
from clue.qiskit import *
from numpy import load, save
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer, qpy
from qiskit.quantum_info import Operator
from time import time

In this document we present the code necessary to transform a ".qasm" file from its original shape to a compressed version that can do the same computations taking into consideration the corresponding lumping. 

The main idea behind this notebook is to take a `.qasm` file, create the corresponding quantum circuit using the unitary matrix associated to it and then compute a lumping for the input $\ket{0}$ which is the usual input for these circuits. Then, after obtaining the new unitary matrix, we will build a new circuit (hopefully smaller) using this matrix and save it as a new `.qasm` file.

Finally, we would like to compare the simulation time. The measurement is slightly tricky in the reduced circuit since we need to know the value of the associated vector that is represented by each base state in the reduced circuit and perform one last measurment there.

## 1. Lumping and storing the reduction

Here we provide a code that allows to reduce a quantum circuit into a (hopefully) smaller representation of it. It stores on memory the reduced version using a unitary matrix and the lumping matrix to recover later the final measurement.

In [2]:
def reduce_circuit(folder: str, filename: str):
    print("Reading the Dynamical System..."); ctime = time()
    ## Reading the ".qasm" circuit
    circuit = DS_QuantumCircuit.from_qasm_file(f"{folder}/{filename}")
    read_time=time()-ctime; 
    ## Computing the lumping for the first state
    print("Computing lumping..."); ctime = time()
    lumped = circuit.lumping([circuit.variables[0]], print_reduction=False, print_system=False)
    lumping_time=time()-ctime
    ## Writing the reduced circuit
    print("Building reduces .qpy..."); ctime = time()
    Ur = lumped.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble); Ur
    Ur = extend_to_power(Ur)
    nqbits = int(log2(Ur.shape[0]))
    q = QuantumRegister(nqbits,'q')
    c = ClassicalRegister(nqbits,'c')
    red_circuit = QuantumCircuit(q,c)
    red_circuit.unitary(Ur, q)
    red_circuit.measure(q, c)
    with open(f"{folder}/reduced/{filename}.qpy", "wb") as f:
        qpy.dump(red_circuit, f)
    reduced_time = time()-ctime
    ## Writing the lumping matrix
    print("Saving the lumping data..."); ctime = time()
    save(f"{folder}/reduced/{filename}.npy", lumped.lumping_matrix.to_numpy(dtype=cdouble))
    matrix_time = time()-ctime
    
    return {"read": read_time, "lumping": lumping_time, "reduced": reduced_time, "matrix": matrix_time}
    

In [6]:
times = reduce_circuit("./circuits", "grover-noancilla_indep_qiskit_7.qasm")
print(times)
print(sum(times.values()))

Reading the Dynamical System...
Computing lumping...
Building reduces .qpy...
Saving the lumping data...
{'read': 1.3891913890838623, 'lumping': 0.8887519836425781, 'reduced': 0.0009846687316894531, 'matrix': 0.0005013942718505859}
2.2794294357299805


In [7]:
times = reduce_circuit("./circuits", "grover-noancilla_indep_qiskit_8.qasm")
print(times)
print(sum(times.values()))

Reading the Dynamical System...
Computing lumping...
Building reduces .qpy...
Saving the lumping data...
{'read': 1.4860002994537354, 'lumping': 2.793416976928711, 'reduced': 0.0011620521545410156, 'matrix': 0.0008666515350341797}
4.2814459800720215


In [8]:
times = reduce_circuit("./circuits", "grover-noancilla_indep_qiskit_9.qasm")
print(times)
print(sum(times.values()))

Reading the Dynamical System...
Computing lumping...
Building reduces .qpy...
Saving the lumping data...
{'read': 6.623856067657471, 'lumping': 11.725273847579956, 'reduced': 0.0010912418365478516, 'matrix': 0.0015690326690673828}
18.351790189743042


In [5]:
times = reduce_circuit("./circuits", "grover-noancilla_indep_qiskit_10.qasm")
print(times)
print(sum(times.values()))

Reading the Dynamical System...
Computing lumping...
Building reduces .qpy...
Saving the lumping data...
{'read': 30.4960196018219, 'lumping': 89.98585891723633, 'reduced': 0.0010149478912353516, 'matrix': 0.005156993865966797}
120.48805046081543


## 2. Simulating a quantum circuit

Here we provide a piece of code that receives how many times to simulate a system ans uses the qiskit framework to do it. It will return the distribution of the outputs

In [9]:
def simulate_qasm(folder: str, filename:str, shots=8192):
    ctime = time()
    ## Reading the quantum circuit
    circuit = QuantumCircuit.from_qasm_file(f"{folder}/{filename}")
    reading_time = time()-ctime; ctime=time()
    ## Simulating the quantum circuit
    backend = Aer.get_backend('aer_simulator')
    job = execute(circuit, backend, shots=shots)
    simulation_time = time()-ctime; ctime=time()
    ## Collecting the data to have proper output
    output = job.result().get_counts()
    output_time = time()-ctime
    
    return output, {"read": reading_time, "simulation": simulation_time, "output": output_time}

In [10]:
res7 = simulate_qasm("./circuits", "grover-noancilla_indep_qiskit_7.qasm")
print(res7[1])
print(sum(res7[1].values()))

{'read': 0.09397745132446289, 'simulation': 0.023434162139892578, 'output': 0.015834569931030273}
0.13324618339538574


In [12]:
res8 = simulate_qasm("./circuits", "grover-noancilla_indep_qiskit_8.qasm")
print(res8[1])
print(sum(res8[1].values()))

{'read': 0.8389289379119873, 'simulation': 0.053495168685913086, 'output': 0.023340702056884766}
0.9157648086547852


In [13]:
res9 = simulate_qasm("./circuits", "grover-noancilla_indep_qiskit_9.qasm")
print(res9[1])
print(sum(res9[1].values()))

{'read': 0.5727097988128662, 'simulation': 0.7066619396209717, 'output': 0.05100226402282715}
1.330374002456665


In [14]:
res10 = simulate_qasm("./circuits", "grover-noancilla_indep_qiskit_10.qasm")
print(res10[1])
print(sum(res10[1].values()))

{'read': 2.288045644760132, 'simulation': 1.2098524570465088, 'output': 0.12458062171936035}
3.622478723526001


## 3. Simulating a reduced quantum circuit

In this case, we assume the `.qasm` file has associated a `.npy` file with the corresponding lumping needed for transforming the measure of the `.qasm` circuit to the actual output of the circuit. This adds a small overload to the simulation. The output will be similar to that in `simulate_qasm`.

In [27]:
def simulate_reduced(folder: str, filename:str, shots=8192):
    ctime = time()
    ## Reading the quantum circuit
    with open(f"{folder}/{filename}", "rb") as f:
        circuit = qpy.load(f)[0]
    reading_time = time()-ctime; ctime=time()    
    ## Simulating the quantum circuit
    backend = Aer.get_backend('aer_simulator')
    job = execute(circuit, backend, shots=shots)
    simulation_time = time()-ctime; ctime=time()
    ## Collecting the data to have proper output
    #### Reading the lumping matrix
    Ur = load(f"{folder}/{filename.removesuffix('.qpy')}.npy")
    nqbits = int(log2(Ur.shape[1]))
    matrix_time = time()-ctime; ctime=time()
    #### Measuring the data from qasm to the original states
    output = dict()
    for (out, times) in job.result().get_counts().items():
        for m,t in repeated_measure(Ur[int(out, 2)], times, out=dict).items():
            output[m] = output.get(m,0) + t
    output = {format(k, f"0{nqbits}b"): v for k,v in output.items()}
    measuring_time = time()-ctime
    output_time = matrix_time + measuring_time
    
    return output, {"read": reading_time, "simulation": simulation_time, "output": {"total": output_time, "matrix": matrix_time, "measure": measuring_time}}

In [28]:
res = simulate_reduced("./circuits/reduced", "grover-noancilla_indep_qiskit_7.qasm.qpy")
print(res[1])
print(sum(el if not isinstance(el, dict) else el["total"] for el in res[1].values()))

{'read': 0.002210855484008789, 'simulation': 0.0032739639282226562, 'output': {'total': 0.010535478591918945, 'matrix': 0.0010366439819335938, 'measure': 0.009498834609985352}}
0.01602029800415039


In [29]:
res[3]

{'01': 8192}

In [ ]:
circuit